# PostgreSQL - Building a database for crime reports

The goal of this guided project is to setup a database from scratch and the Boston crime data into it.

We will create two user groups:

readonly: Users in this group will have permission to read data only.
readwrite: Users in this group will have permissions to read and alter data but not to delete tables.

## Creating the database and the schema

Create a database named crime_db and a schema named crimes for storing the tables for containing the crime data.

In [1]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True
cur=conn.cursor()
cur.execute("CREATE DATABASE crime_db")
conn.close()

ProgrammingError: database "crime_db" already exists


In [2]:
conn = psycopg2.connect("dbname=crime_db user=dq")
conn.autocommit = True
cur=conn.cursor()
cur.execute("CREATE SCHEMA crimes")

ProgrammingError: schema "crimes" already exists


## Obtaining the Column Names and Sample

Obtain the header row and assign it to a variable named col_headers and obtain the first data row and assign it to a variable named first_row.

In [3]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Creating a function for analyzing column values

Create a function get_col_set that given a CSV file name and a column index computes the set of all distinct values in that column.

Use the function on each column to evaluate which columns have a lot of different values. Columns with a limited set of possible values are good candidates for enumerated datatypes.

In [4]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values =  get_col_set('boston.csv', i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Analyzing the maximum length of the description column
Use the get_col_set function to compute the maximum description length to decide an appropriate length for that field.

In [5]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [6]:
descriptions = get_col_set("boston.csv", 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Creating the table
We have create an enumerated datatype named weekday for the day_of_the_week since there there only seven possible values.

For the incident_number we have decided to user the type INTEGER and set it as the primary key. The same datatype was also used to represent the offense_code.

Since the description has at most 58 character we decided to use the datatype VARCHAR(100) for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The date was represented as the DATE datatype. Finally, for the latitude and longitude we used DECIMAL datatypes.

In [7]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We will use the same names for the column headers.

The number of different values of each column was:

incident_number 298329  
offense_code       219  
description        239  
date              1177  
day_of_the_week      7  
lat              18177  
long             18177  

From the result of printing first_row we see that kind of data that we have are:

integer numbers  
integer numbers  
string  
date  
string  
decimal number  
decimal number  

Only column day_of_the_week has a small range of values so we will only create an enumerated datatype for this column. Column offense_code is also a good candidate since there is probably a limited set of possible offense codes.

We saw that the offense_code column has size at most 59. To be on the safe side we will limit the size of the description to 100 and use the VARCHAR(100) datatype.

The lat and long column see to need to hold quite a lot of precision so we will use the decimal type.

In [17]:
#cur.execute("""DROP TYPE weekday""")
#cur.execute("""
    #CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
#""")

cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ProgrammingError: relation "boston_crimes" already exists


## Load the data into the table
We used the copy_expert to load the data as it is very fast and very succinct to use.

In [24]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

cur.execute("select * from crimes.boston_crimes;")
print(len(cur.fetchall()))

298329


## Revoke public privileges
We revoke all privileges of the public public group on the public schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the public schema.

We also need to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [30]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only group
We create a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal wit tables privileges by granting SELECT. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new table that are created on the crimes schema. As we mentioned, by default not privileges are given. However we change is so that by default any user in the readonly group can issue select commands.

In [31]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")

## Creating the read-write group¶
We create a readwrite group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal wit tables privileges by granting SELECT, INSERT, UPDATE and DELETE. As before we change the default privileges so that user in the readwrite group have these privileges if we ever create a new table on the crimes schema.

In [32]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

## Creating one user for each group¶
We create a user named data_analyst with password secret1 in the readonly group.

We create a user named data_scientist with password secret2 in the readwrite group.

In [33]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist")

In [38]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
